In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from urllib.error import HTTPError

In [ ]:
'''
Use the function below to create a dataframe of articles from datasets
containing a link to the article.

inputFile: Path to the input dataset
outputFile: Path for output dataframe to be saved
range: Number of articles to be extracted (for testing)
'''

def extractArticles(inputFile, outputFile, range=-1):
  dataset_df = pd.read_csv(inputFile)
  df = pd.DataFrame(columns=['Link', 'Article'])
  count = 0

  for _, row in dataset_df.iterrows():
    if range != -1 and count == range:
      break
    link = str(row['news_url'])
    if link[0:4] != 'http':
      link = 'https://' + link
    count = count + 1
    try:
      response = urllib.request.urlopen(link, timeout=20)
      html = response.read()

      soup = BeautifulSoup(html, 'html.parser')

      for table in soup.find_all('table'):
        table.decompose()
      paragraphs = [data.get_text() for data in soup.find_all("p")]
      article = ' '.join(paragraphs)
      article = ' '.join(article.split())

    except HTTPError as e:
        if e.code == 404:
            article = "Not found"
        else:
            article = f"Error: {e}"
    except Exception as e:
        article = f"Error: {e}"
    except HTTPError as e:
        if e.code == 404:
            article = "Not found"
        else:
            article = f"HTTP Error: {e}"
    except ConnectionError as e:
        article = f"Connection Error: {e}"
    except Exception as e:
        article = f"Error: {e}"

    df = df.append({'Link': link, 'Article': article}, ignore_index=True)

  df.to_csv(outputFile, index=False)